In [1]:
student_number = '400104801'
Name = 'Mehran'
Last_Name = 'Bakhtiari'

# Deep Reinforcement Learning

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [3]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 17.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [5]:
env_name = 'CartPole-v1'

env = gym.make(env_name)

obs = env.reset()

frames = []

for _ in range(500):
    action = env.action_space.sample()

    obs, reward, done, _ = env.step(action)

    frames.append(env.render(mode='rgb_array'))

    if done:
        env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initiali

In [6]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.

You need to implement the convolutional neural network.


In [40]:
class ActorCritic(nn.Module):
    def __init__(self, input_dim, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc_actor = nn.Linear(hidden_size, num_outputs)
        self.fc_critic = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_probs = F.softmax(self.fc_actor(x), dim=-1)
        value = self.fc_critic(x)
        return action_probs, value

## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [50]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        input_dim = env.observation_space.shape[0]
        num_outputs = env.action_space.n

        self.policy_net = ActorCritic(input_dim, hidden_size, num_outputs).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        self.critic_loss = nn.MSELoss()

    def choose_action(self, state):
        state = torch.FloatTensor(state).to(self.device)
        action_probs, _ = self.policy_net(state)
        action_dist = Categorical(action_probs)
        action = action_dist.sample()
        return action.item()

    def compute_returns(self, rewards):
        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns).to(self.device)
        return returns

    def train(self):
        episode_rewards = []
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                state = self.env.reset()
                log_probs = []
                values = []
                rewards = []
                episode_reward = 0

                for step in range(self.max_steps):
                    state = torch.FloatTensor(state).to(self.device)
                    action_probs, value = self.policy_net(state)
                    action_dist = Categorical(action_probs)
                    action = action_dist.sample()
                    next_state, reward, done, _ = self.env.step(action.cpu().numpy())

                    log_prob = action_dist.log_prob(action).unsqueeze(0)
                    log_probs.append(log_prob)
                    values.append(value.detach())
                    rewards.append(reward)

                    state = next_state
                    episode_reward += reward

                    if done:
                        break

                episode_rewards.append(episode_reward)

                log_probs = torch.cat(log_probs, dim=0)
                returns = self.compute_returns(rewards)
                values = torch.cat(values)
                advantages = returns - values

                actor_loss = -(advantages.detach() * log_probs).mean()
                critic_loss = self.critic_loss(values, returns)
                loss = actor_loss + critic_loss

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                pbar.set_description(f"Episode {episode + 1}, Reward: {episode_reward}")

        self.env.close()
        return episode_rewards

Define the model and set hyperparameters.

In [53]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

num_episodes = 3000
max_steps = 500
lr = 1e-3
hidden_size = 256

a2c_model = A2CAgent(env, num_episodes=num_episodes, max_steps=max_steps, gamma=0.99, lr=lr, hidden_size=hidden_size)

Train the model.

In [54]:
rewards = a2c_model.train()

Episode 3000, Reward: 500.0: 100%|██████████| 3000/3000 [18:48<00:00,  2.66it/s]


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [60]:
env = gym.make(env_name)
model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    done = False

    while not done:
        frames.append(env.render(mode='rgb_array'))

        action = model.choose_action(state)

        next_state, reward, done, _ = env.step(action)

        state = next_state
        episode_reward += reward

    episode_rewards.append(episode_reward)
    print(f"Episode {i+1} Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
avg_reward = np.mean(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {avg_reward}")

imageio.mimsave('./cartpole.mp4', frames, fps=25)

Episode 1 Reward: 500.0
Episode 2 Reward: 471.0
Episode 3 Reward: 500.0
Episode 4 Reward: 500.0
Episode 5 Reward: 500.0
Episode 6 Reward: 500.0
Episode 7 Reward: 500.0
Episode 8 Reward: 500.0
Episode 9 Reward: 500.0


Episode 10 Reward: 500.0
Average Reward over 10 episodes: 497.1


In [61]:
show_video('./test.mp4')